In [68]:
# from libraries
import pandas as pd
import dask.dataframe as dd
# from files
import params
import auth
import get_data_util
import get_sp500_util as sp500

In [69]:
import requests
from requests.adapters import HTTPAdapter
from urllib3.util import Retry
import pandas as pd
import dask.dataframe as dd
from io import StringIO
import boto3
import quandl
import time

# from files:
import auth
import params

In [64]:
ticker_list = sp500.get_sp_500(url=sp500.url, headers=auth.user_agent)

In [74]:
alpha_vantage_base = "https://www.alphavantage.co"  # url base
param_dict_names = [name for name in dir(params) if ('params' in name)]
param_dict_list = [getattr(params, name) for name in param_dict_names]

In [75]:
stock_info_list = []
for stock_symbol in ticker_list[:3]:
    stock_indicators = []
    for i in range(len(param_dict_list)):
            param_dict_list[i]["symbol"] = stock_symbol
            session = requests.Session()
            retry = Retry(connect=5, backoff_factor=0.5)
            adapter = HTTPAdapter(max_retries=retry)
            session.mount("http://", adapter)
            session.mount("https://", adapter)
            r = session.get(alpha_vantage_base + '/query', params=param_dict_list[i], headers=auth.user_agent)
            df = pd.read_csv(StringIO(r.content.decode('utf-8')), sep=",", engine="python")
            df.set_index(df.columns[0], inplace=True)
            ddf = dd.from_pandas(df, npartitions=4)
            del df
            ddf.columns = [param_dict_list[i]["function"] + "_" + str(column) for column in ddf.columns]
            if param_dict_list[i]["function"] == "TIME_SERIES_DAILY_ADJUSTED":
                ddf["stock_symbol"] = stock_symbol
            stock_indicators.append(ddf)
            print("Successfully extracted {}!".format(stock_symbol+"_"+param_dict_names[i].replace("_params","")))
    stock_info = dd.concat(stock_indicators, axis=1)
    stock_info_list.append(stock_info)
    concat_data = dd.concat(stock_info_list, axis=0)

Successfully extracted AVB_ad!
Successfully extracted AVB_aroon!
Successfully extracted AVB_bbands_close!
Successfully extracted AVB_bbands_high!
Successfully extracted AVB_bbands_low!
Successfully extracted AVB_bbands_open!
Successfully extracted AVB_cci!
Successfully extracted AVB_ema_close!
Successfully extracted AVB_ema_high!
Successfully extracted AVB_ema_low!
Successfully extracted AVB_ema_open_200!
Successfully extracted AVB_macd_close!
Successfully extracted AVB_macd_high!
Successfully extracted AVB_macd_low!
Successfully extracted AVB_macd_open!
Successfully extracted AVB_obv!
Successfully extracted AVB_rsi_close!
Successfully extracted AVB_rsi_high!
Successfully extracted AVB_rsi_low!
Successfully extracted AVB_rsi_open!
Successfully extracted AVB_stoch!
Successfully extracted AVB_stock_time_series_adjusted!
Successfully extracted EW_ad!
Successfully extracted EW_aroon!
Successfully extracted EW_bbands_close!
Successfully extracted EW_bbands_high!
Successfully extracted EW_bb

In [45]:
ddf0 = dd.from_pandas(stock_indicators[0], npartitions=4)
ddf1 = dd.from_pandas(stock_indicators[1], npartitions=4)
ddf2 = dd.from_pandas(stock_indicators[2], npartitions=4)

In [55]:
param_dict_list[0]

{'function': 'AD',
 'interval': 'daily',
 'datatype': 'csv',
 'apikey': 'RPSX222Y3CANA3IV',
 'symbol': 'EW'}

In [56]:
ddf0.columns = [param_dict_list[0]["function"]+"_"+str(column) for column in ddf0.columns]
ddf1.columns = [param_dict_list[1]["function"]+"_"+str(column) for column in ddf1.columns]
ddf2.columns = [param_dict_list[1]["function"]+"_"+str(column) for column in ddf2.columns]

In [58]:
a = dd.concat([ddf0,ddf1,ddf2], axis=1)

In [61]:
dd.concat([a, a, a], axis=0)

,AD_Chaikin A/D,AROON_Aroon Down,AROON_Aroon Up,AROON_Real Lower Band,AROON_Real Middle Band,AROON_Real Upper Band
npartitions=36,,,,,,
,float64,float64,float64,float64,float64,float64
,...,...,...,...,...,...
...,...,...,...,...,...,...
,...,...,...,...,...,...
,...,...,...,...,...,...


In [70]:
alpha_vantage_data = get_data_util.get_alpha_vantage_data(ticker_list[:2])

Getting data for AVB...
Process Begin...
Successfully extracted AVB_ad!
Process Begin...
Successfully extracted AVB_aroon!
Process Begin...
Successfully extracted AVB_bbands_close!
Process Begin...
Successfully extracted AVB_bbands_high!
Process Begin...
Successfully extracted AVB_bbands_low!
Process Begin...
Successfully extracted AVB_bbands_open!
Process Begin...
Successfully extracted AVB_cci!
Process Begin...
Successfully extracted AVB_ema_close!
Process Begin...
Successfully extracted AVB_ema_high!
Process Begin...
Successfully extracted AVB_ema_low!
Process Begin...
Successfully extracted AVB_ema_open_200!
Process Begin...
Successfully extracted AVB_macd_close!
Process Begin...
Successfully extracted AVB_macd_high!
Process Begin...
Successfully extracted AVB_macd_low!
Process Begin...
Successfully extracted AVB_macd_open!
Process Begin...
Successfully extracted AVB_obv!
Process Begin...
Successfully extracted AVB_rsi_close!
Process Begin...
Successfully extracted AVB_rsi_high!
Pr

In [71]:
alpha_vantage_data

,stock_symbol
npartitions=208,
,object
,...
...,...
,...
,...


In [72]:
alpha_vantage_data.compute()

,stock_symbol
"1999-09-10,0.0000",NaN
"1999-09-13,47700.0000",NaN
"1999-09-14,6085.6000",NaN
"1999-09-15,-33884.6128",NaN
"1999-09-16,-69695.8128",NaN
"1999-09-17,55704.1872",NaN
"1999-09-20,29356.9029",NaN
"1999-09-21,13487.3029",NaN
"1999-09-22,-109807.3590",NaN
"1999-09-23,-189707.3590",NaN
